# AR6/WGI Chapter 8

# Build a figure showing long-term change for 4 variables for one SSP

## S.Sénési - sept 2019 to march 2021

### A few commands below are specific to the Notebook environment, and can be safely commented out

## Default settings (some may be overriden by Papermill - this would show in next cell in the execution output notebook)

In [ ]:
do_test              = True

figure_name          = "FigTS6x"
# Version number will be a suffix for automatic figure filename. Use e.g. "_V1" for legibility
version              = ""
scheme               = "AR6S"  # Hatching scheme
confidence_factor    = 1.645  # For AR6 comprehensive scheme : Multiplicative factor applied to control run 
                              # variability for deciding a change is significant (besides sqrt(2))
sign_threshold       = 0.8    # For AR6 simple scheme : threshold on cross-model change sign agreeement fraction
same_models_for_var  = False
manual_title         = None  # Will replace automatic title if not None

#See doc for data_versions in sibling directory data_versions
data_versions_tag   = "20210201_derived"
data_versions_dir   = "/home/ssenesi/CAMMAC/select_data_versions"
excluded_models     = { }
included_models     = None
variability_excluded_models = {}
variability_models  = None
#
experiment          = "ssp245"
proj_period         = "2081-2100"
#
ref_experiment      = "historical"
ref_period          = "1995-2014" 
#
season              = "ANN"        # Other choices do not make sense for some variables (yet)
field_type          = "means_rchange" # Ranges in figures are not tuned for other choices ...

# Next value is used as a floor value of reference field for computing relative changes 
# of some variables
z1=0.1/(24*3600) # 0.1 mm/day, in kg m2 s-1
#
# If pre-computed fields for this SSP, season and projection_period are available, should we use it ?
# Set it to False for recomputing and printing fields and field changes statistics
# No problem if set to True and pre-computed fields does not exist
use_cached_proj_fields = True   
drop_old_figures       = False
#
outdir              = "./figures"
cache_dir           = "./cache"
variab_sampling_args= {"house_keeping":True,"compute":True,"detrend":True,"shift":100,"nyears":20,"number":20}
figure_details      = {"page_width":2450,"page_height":3444,"pt":48, "ybox":133,"y":52,"insert_width":700}
common_grid         = "r360x180"
antarctic_mask      = "/home/ssenesi/CAMMAC/data/mask_hide_antarctic_360x180.nc"
#
derived_variables_pattern  = "/data/ssenesi/CMIP6_derived_variables/${variable}"
derived_variables_pattern += "/${variable}_${table}_${model}_${experiment}_${realization}_${grid}_${version}_${PERIOD}.nc"
derived_variable_table='yr'
#

# Location of libraries
#######################
# Climaf version >= 1.2.13 (see https://climaf.readthedocs.io)
climaf_lib          = "/home/ssenesi/climaf_installs/climaf_running" 
# AR6/WGI/chapter8 CliMAF-based package
CAMMAC              = "/home/ssenesi/CAMMAC"

In [ ]:
if do_test :
    version             = "_test"
    proj_period         = "2099-2100"
    ref_period          = "2012-2014" 
    included_models     = ["CNRM-CM6-1"]

## Loading libraries

In [ ]:
import sys, os

# Climaf setup (version >= 1.2.13 - see https://climaf.readthedocs.io)
sys.path=[climaf_lib] + sys.path 
from climaf.api import *

# Climaf settings
from climaf.cache import stamping
climaf.cache.stamping=False

# AR6 figures project package
sys.path.append(CAMMAC    ) 
from CAMMAClib.changes     import AR6_change_figure_with_caching
from CAMMAClib.ancillary   import prettier_label
from CAMMAClib.mips_et_al  import TSU_metadata, read_versions_dictionnary

In [ ]:
# Fix sign or scale issue with some models for a few variables
calias('CMIP6','evspsbl',scale=-1,\
       conditions={"model":["CAMS-CSM1-0","EC-Earth3","EC-Earth3-Veg","EC-Earth3-LR","EC-Earth3-Veg-LR"]})
calias('CMIP6','pr',scale=1000.,conditions={"model" : "CIESM"})
calias('CMIP6','mrso',scale=1000.,conditions={"model" : "CIESM"})
calias('CMIP6','mrsos',scale=100.,conditions={"model" : "FGOALS-f3-L"})

# Define P-E
derive('CMIP6', 'P-E','minus','pr','evspsbl')

# Define location of derived variables if needed (e.g. yearly stats of daily precip)
if derived_variables_pattern is not None :
    climaf.dataloc.dataloc(project='CMIP6', organization='generic', 
                           url=derived_variables_pattern, table=derived_variable_table)

### If using a notebook, use wide display

In [ ]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))

## Next function combines global variables and its own arguments for calling AR6_change_figure_with_caching

In [ ]:
def afigure(variable, title, panel, plot_args,  derivation_label = "plain",field_type = field_type, 
            mask = None, table = "Amon", data_versions_tag = data_versions_tag, threshold=None, deep=None,
            read = use_cached_proj_fields ):

    global metadata
    
    key=variable
    if table=="day" :
        key=variable+"_day"
    exclude=excluded_models.get(key,[])
    vexclude=variability_excluded_models.get(key,[])
    #
    fig_file,fig,dic,variab_models,models = AR6_change_figure_with_caching(
        ref_period = ref_period, proj_period = proj_period, 
        variable = variable, table = table, ref_experiment = ref_experiment,
        experiment = experiment, season = season,
        derivation_label = derivation_label, 
        field_type = field_type,
        title = panel+") "+ title,
        custom_plot = plot_args, labelbar = "True", 
        data_versions_tag = data_versions_tag, data_versions_dir = data_versions_dir,
        excluded_models = exclude, models = included_models,
        variability_excluded_models = vexclude, 
        variability_models = variability_models,
        outdir = outdir, 
        #
        common_grid = common_grid, mask=mask,
        variab_sampling_args = variab_sampling_args,
        cache_dir = cache_dir, read = read, write = True, 
        print_statistics = True , deep = deep,  drop=drop_old_figures,
        threshold = threshold,
        same_models_for_variability_and_changes=same_models_for_var,
        scheme=scheme,
        low_change_agree_threshold=confidence_factor,
        change_sign_agree_threshold = sign_threshold
        )
    metadata += TSU_metadata([ref_experiment,experiment],models       ,variable,table,data_versions,panel)
    metadata += TSU_metadata(["piControl"]              ,variab_models,variable,table,data_versions,panel)
    return fig,dic

## Compute figures for the four variables

In [ ]:
col40={"colors":"-40 -30 -20 -10 -5   0   5   10  20 30 40 "}
col10={"colors":"-10 -8 -6 -4 -2 0 2 4 6 8 10"}
data_versions=read_versions_dictionnary(data_versions_tag, data_versions_dir)
dic=dict()

if True :
    metadata=""
    #
    l1c1,dic[("pr","plain")]      = afigure("pr"     ,"Precipitation"               ,"a",col40, threshold=z1)
    
    l1c2,dic[("evspsbl","plain")] = afigure("evspsbl","Evapotranspiration"          ,"b",col40, threshold=z1)
    
    #
    l2c1,dic[("mrro","plain")]    = afigure("mrro"   ,"Runoff"                      ,"c",col40, table="Lmon",mask=antarctic_mask, threshold=z1)
    
    l2c2,dic[("mrsos","plain")]   = afigure("mrsos"  ,"Surface soil moisture"       ,"d",col10, table="Lmon",mask=antarctic_mask)
    
    import os.path
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    with open("%s/%s%s_md.txt"%(outdir,figure_name,version),"w") as f:
        f.write(metadata)


## Assemble plots

In [ ]:
if manual_title is None :
    title="Long term water cycle variables changes for %s (%s vs %s)"%(prettier_label.get(experiment,experiment),proj_period,ref_period)
else:
    title=manual_title

if scheme=="AR6S" :
    insert=CAMMAC+"/CAMMAClib/captions/AR6_simple_hatching_wide.png"
else :
    insert=CAMMAC+"/CAMMAClib/captions/AR6_comprehensive_hatching.png"
    

page=cpage([[l1c1,l1c2],[l2c1,l2c2]], title=title, insert=insert, font="Liberation-Sans",**figure_details)
outfile="change_map_4vars_1SSP_%s_%s_%s%s.png"%(experiment,proj_period,data_versions_tag,version)
if drop_old_figures :
    cdrop(page)
cfile(page,outdir+"/"+outfile)
os.system("cd %s ; ln -sf %s %s%s.png"%(outdir,outfile,figure_name,version))
#
small=outfile.replace(".png",".small.png")
os.system("cd %s ; convert -geometry 50%% %s %s"%(outdir,outfile,small))
os.system("cd %s ; ln -sf %s %s%s_small.png"%(outdir,small,figure_name,version))


## If using a notebook , display result

In [ ]:
Image(outdir+"/"+outfile,width=800)